In [2]:
import dgl
import torch as th

Using backend: pytorch


# 图

In [3]:
# 有向图
# 边 0->1, 0->2, 0->3, 1->3
u, v = th.tensor([0,0,0,1]), th.tensor([1,2,3,3])
g = dgl.graph((u, v))
print('g', g)
print('nodes', g.nodes())
print('edges',g.edges())

g Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})
nodes tensor([0, 1, 2, 3])
edges (tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]))


In [4]:
# 无向图
bg = dgl.to_bidirected(g)
bg.edges()


(tensor([0, 0, 0, 1, 1, 2, 3, 3]), tensor([1, 2, 3, 0, 3, 0, 0, 1]))

In [20]:
# 节点特征，其中x特征为3维向量，y特征为5维向量
g.ndata['x'] = th.ones(g.num_nodes(), 3)
g.ndata['y'] = th.randn(g.num_nodes(), 5)
# 边特征，x特征是标量，y特征是2维向量
g.edata['x'] = th.ones(g.num_edges(), dtype=th.int32)
g.edata['y'] = th.ones(g.num_edges(), 2, dtype=th.int32)
print(g)
print('feature of vertex 1 =x', g.ndata['x'][1])
print('feature of edge 0 and 3 =x', g.edata['x'][th.tensor([0, 3])])
print('feature of edge 1 and 2 =y', g.edata['y'][th.tensor([1, 2])])
# 注意：需要一次性赋值所有点或边，无法赋值子图

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'y': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(), dtype=torch.int32), 'y': Scheme(shape=(2,), dtype=torch.int32)})
feature of vertex 1 =x tensor([1., 1., 1.])
feature of edge 0 and 3 =x tensor([1, 1], dtype=torch.int32)
feature of edge 1 and 2 =y tensor([[1, 1],
        [1, 1]], dtype=torch.int32)


In [24]:
# 对于加权图，可以将权重储存为一个边特征
weight = th.tensor([0.1,0.6,0.8,0.2])
g.edata['w'] = weight
print(g)

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'y': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(), dtype=torch.int32), 'y': Scheme(shape=(2,), dtype=torch.int32), 'w': Scheme(shape=(), dtype=torch.float32)})


In [159]:
# 异构图
graph_data = {
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
}
# 不同的节点类型有各自的单独一套ID
# 这个字典表示 drug类型节点有0，1，2共三个，disease类型节点有0，1，2共3个，gene类型节点有0，1，2，3共四个
# gene节点中只有g1和g2有连接边，其他两个没有
g = dgl.heterograph(graph_data)
g.nodes['drug'].data['xx'] = th.ones(3, 1)
print(g.nodes['drug'])
print(g)
print(g.metagraph().edges())

NodeSpace(data={'xx': tensor([[1.],
        [1.],
        [1.]])})
Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])
[('drug', 'drug'), ('drug', 'gene'), ('drug', 'disease')]


In [161]:
print(g.canonical_etypes)
for c_etype in g.canonical_etypes:
    print(c_etype)
print(g.nodes['drug'])
with g.local_scope():
    g.nodes['drug'].data['h'] = th.tensor(th.FloatTensor(3,1))
    print(g.nodes['drug'])
print(g.nodes['drug'])

[('drug', 'interacts', 'drug'), ('drug', 'interacts', 'gene'), ('drug', 'treats', 'disease')]
('drug', 'interacts', 'drug')
('drug', 'interacts', 'gene')
('drug', 'treats', 'disease')
NodeSpace(data={'xx': tensor([[1.],
        [1.],
        [1.]])})
NodeSpace(data={'xx': tensor([[1.],
        [1.],
        [1.]]), 'h': tensor([[0.0000e+00],
        [1.8980e+01],
        [1.1314e-14]])})
NodeSpace(data={'xx': tensor([[1.],
        [1.],
        [1.]])})


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [47]:
# 各种类型的节点数量
print('all nodes', g.num_nodes())
print('drug nodes', g.num_nodes('drug'))
# 如果使用nodes需要指定类型
print('gene nodes', g.nodes('gene'))
print('all edges', g.num_edges())

all nodes 10
drug nodes 3
gene nodes tensor([0, 1, 2, 3])
all edges 5


In [49]:
# 获取额点节点/边的特征
g.nodes['drug'].data['hv'] = th.ones(3,1)
print(g)
print(g.nodes['drug'].data['hv'])

Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])
tensor([[1.],
        [1.],
        [1.]])


In [55]:
# 将异构图转换为同构图
hg = dgl.to_homogeneous(g)
print(hg)

Graph(num_nodes=10, num_edges=5,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)})


# 消息传递

In [134]:
# 建立一个带属性的图
u, v = th.tensor([0,0,0,1]), th.tensor([1,2,3,3])
g = dgl.graph((u, v), num_nodes=5)
g.ndata['nx'] = th.ones(g.num_nodes(), 3)
g.ndata['ny'] = th.randn(g.num_nodes(), 3)
g.edata['ex'] = th.ones(g.num_edges(), dtype=th.int32)
g.edata['ey'] = th.ones(g.num_edges(), 2, dtype=th.int32)
print(g)

Graph(num_nodes=5, num_edges=4,
      ndata_schemes={'nx': Scheme(shape=(3,), dtype=torch.float32), 'ny': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'ex': Scheme(shape=(), dtype=torch.int32), 'ey': Scheme(shape=(2,), dtype=torch.int32)})


In [135]:
# 消息函数，即边的信息计算
fn_mas = dgl.function.u_add_v('nx','ny','exx')
g.apply_edges(fn_mas)
# 以上函数是将source点的nx属性与distination点的ny属性相加，然后赋值给名为exx的属性
print(g)
print(g.ndata)
print('exx',g.edata['exx'])

Graph(num_nodes=5, num_edges=4,
      ndata_schemes={'nx': Scheme(shape=(3,), dtype=torch.float32), 'ny': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'ex': Scheme(shape=(), dtype=torch.int32), 'ey': Scheme(shape=(2,), dtype=torch.int32), 'exx': Scheme(shape=(3,), dtype=torch.float32)})
{'nx': tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]), 'ny': tensor([[ 1.9593, -0.2172, -0.3068],
        [-0.3790,  0.0845, -0.3156],
        [-0.2208,  0.2509, -0.4487],
        [-1.1195, -0.4370, -0.0675],
        [-1.9719, -0.3470, -0.1099]])}
exx tensor([[ 0.6210,  1.0845,  0.6844],
        [ 0.7792,  1.2509,  0.5513],
        [-0.1195,  0.5630,  0.9325],
        [-0.1195,  0.5630,  0.9325]])


In [136]:
# 聚合函数，即将一个节点处所有边的信息进行聚合计算
fn_red = dgl.function.sum('exx','nxx')
g.update_all(fn_mas, fn_red)
print(g)

Graph(num_nodes=5, num_edges=4,
      ndata_schemes={'nx': Scheme(shape=(3,), dtype=torch.float32), 'ny': Scheme(shape=(3,), dtype=torch.float32), 'nxx': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'ex': Scheme(shape=(), dtype=torch.int32), 'ey': Scheme(shape=(2,), dtype=torch.int32), 'exx': Scheme(shape=(3,), dtype=torch.float32)})


In [142]:
linear_src = th.nn.Parameter(th.FloatTensor(size=(1,3)))
linear_dst = th.nn.Parameter(th.FloatTensor(size=(1,3)))
out_src = g.ndata['nxx'] * linear_src
out_dst = g.ndata['nxx'] * linear_dst
g.srcdata.update({'nxx_src':out_src})
g.dstdata.update({'nxx_dst':out_dst})
g.apply_edges(dgl.function.u_add_v('nxx_src','nxx_dst','out'))
print(g)

Graph(num_nodes=5, num_edges=4,
      ndata_schemes={'nx': Scheme(shape=(3,), dtype=torch.float32), 'ny': Scheme(shape=(3,), dtype=torch.float32), 'nxx': Scheme(shape=(3,), dtype=torch.float32), 'nxx_src': Scheme(shape=(3,), dtype=torch.float32), 'nxx_dst': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'ex': Scheme(shape=(), dtype=torch.int32), 'ey': Scheme(shape=(2,), dtype=torch.int32), 'exx': Scheme(shape=(3,), dtype=torch.float32), 'out': Scheme(shape=(3,), dtype=torch.float32)})
